<a href="https://colab.research.google.com/github/dhenypatungka/StableDiffusionNotebook/blob/main/SDXL%2BREFINER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### ♍ **Install Diffusers**
#@markdown - Pastikan memilih T4 GPU untuk hardware accelerator.
#@markdown - Tekan gambar folder disamping untuk  melihat hasil generate.

!mkdir "/content/Results"
basePath = "/content/Results"
%cd {basePath}

!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

from IPython.display import clear_output
clear_output()

In [ ]:
#@markdown ### ♍ **SDXL + Refiner**

from diffusers import DiffusionPipeline, AutoencoderKL
import torch

vae = AutoencoderKL.from_pretrained(
    "madebyollin/sdxl-vae-fp16-fix",
    torch_dtype=torch.float16
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
)
pipe.to("cuda");
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

from IPython.display import clear_output
clear_output()

prompt = "an image of a person with a flowing, sculptural hairstyle and garment that echoes the movement of water. The hair and attire should be crafted with smooth, swirling patterns that extend and blend into each other, in a seamless and organic fashion. The color scheme should be a red gradient, set against a majestic red background to emphasize the subject. The subject's pose should be elegant and poised, with a profile view showcasing refined features and a calm, contemplative expression. Add a detailed, ornate earring to the subject to bring an additional element of sophistication. The lighting should cast soft shadows to define the swirling forms, creating a luxurious and modern aesthetic, detailed acrylic, grunge, intricate complexity, rendered in unreal engine, photorealistic, impression, masterpiece, best quality, highly detailed, intricate, beautiful detailed, refiner, high contrast, high detail, vivid color, 8k" # @param {'type' : 'string'}
Negative = "bad anatomy, lowres, normal quality, grayscale, worstquality, watermark, bad proportions, out of focus, long neck, deformed, mutated, mutation, disfigured, poorly drawn face, skin blemishes, skin spots, acnes, missing limb, malformed limbs, floating limbs, disconnected limbs, extra limb, extra arms, mutated hands, poorly drawn hands, malformed hands, mutated hands and fingers, bad hands, missing fingers, fused fingers, too many fingers, extra legs, bad feet, cross-eyed,(distorted, :1.3), (:1.4), low quality, camera, BadDream, UnrealisticDream, bad-hands-5, BadNegAnatomyV1-neg, EasyNegative, FastNegativeV2, bad-picture-chill-75v" # @param {'type' : 'string'}

high_noise_frac = 0.8
#@markdown <p>
iMages = 1 #@param {type:"slider", min:1, max:4, step:1}
Guidance = 7 #@param {type:"slider", min:0, max:15, step:1}
Width = 768 #@param {type:"slider", min:256, max:1024, step:128}
Height = 768 #@param {type:"slider", min:256, max:1024, step:128}
Steps = 20 #@param {type:"slider", min:1, max:100, step:1}

image = pipe(
    prompt=prompt,
    negative_prompt=Negative,
    guidance_scale=Guidance,
    num_images_per_prompt=iMages,
    width=Width,
    height=Height,
    num_inference_steps=Steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

image = refiner(
    prompt=prompt,
    negative_prompt=Negative,
    guidance_scale=Guidance,
    num_images_per_prompt=iMages,
    width=Width,
    height=Height,
    num_inference_steps=Steps,
    denoising_start=high_noise_frac,
    image=image,
).images

import datetime
e = datetime.datetime.now()
for i, image in enumerate(image):
  image.save(f"/content/Results/dp_{e.hour}:{e.minute}:{e.second}_{i}.png")

from IPython.display import clear_output
clear_output()

In [ ]:
#@markdown ### ♍ **Restart Session**
#@markdown - Restart session jika sering terjadi crash.
import os
os.kill(os.getpid(), 9)